## Catalog cross-matching

Some conventions I use:
- The first letter of `astropy.table` object variable names are capitalized
- First letters of `pandas.DataFrame` names are lowercase
- CXO, CSC = Chandra X-ray Observatory, Chandra Source Catalog
- XMM, SSC = XMM-Newton, Serendipitous Source Catalog
- XRT, XPS = (Swift) X-ray Telescope, XRT Point Source (catalog)
- BeSS = Be Star Spectra database

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, astropy.units as u, pickle, seaborn as sns
from mpl_toolkits import mplot3d
from scipy import stats
from PyAstronomy import pyasl
from astropy.io import fits, ascii as ascii_io, votable
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad

from tools import hms_to_dec, dms_to_dec

## Pseudo-manually matched

In [74]:
pd.read_csv("BeStarXrayCatalog.csv")

,BeSS #,Name,RA,Dec,Vmag,SpecType,Simbad classification,Catalog memberships,Literature classification,Reference,...,Flux9,e_Flux9,HR1,e_HR1,HR2,e_HR2,HR3,e_HR3,HR4,e_HR4
0,52,RX J0049.7-7323,12.424958,-73.387419,14.980,B2Ve,HMXB,"Gaia, Chandra, XMM, Swift",Be/XRB,http://articles.adsabs.harvard.edu/pdf/2000A%2...,...,1.365720e-13,3.288130e-15,0.781369,0.040622,0.672771,0.018267,0.141023,1.67773e-02,-0.024442,0.018696
1,60,DZ Tuc,12.686208,-73.268100,15.440,Be,HMXB,"Gaia, Chandra, XMM, Swift",Be/Xray pulsar,https://ui.adsabs.harvard.edu/abs/2006Ap%26SS....,...,1.940010e-13,2.836230e-15,0.850844,0.026943,0.628668,0.016180,0.120864,1.47305e-02,-0.073112,0.017265
2,66,SMC 25,12.966708,-73.176100,14.450,Be,HMXB,"Gaia, Chandra, XMM, Swift",Be/Xray pulsar,https://ui.adsabs.harvard.edu/abs/2006AJ....13...,...,5.780840e-13,5.192510e-15,0.664199,0.013051,0.297753,0.010520,-0.034882,9.86319e-03,-0.193246,0.011405
3,67,2E 0050.1-7247,12.971542,-72.530139,14.870,Be,HMXB,"Gaia, Chandra, Swift",Be/XRB,https://ui.adsabs.harvard.edu/abs/1997ApJ...48...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,69,X SMC X-3,13.024042,-72.434219,14.910,O9IVe,NaN,"XMM, Swift",unconfirmed,no mentions,...,3.166140e-13,3.549050e-15,0.662555,0.004566,0.442941,0.003273,-0.007716,3.12410e-03,-0.290036,0.003289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2185,BD+53 2790,331.984333,54.518444,9.930,O9.5IIIe,HMXB,"Gaia, XMM, Swift",O dwarf/accreting magnetar,https://ui.adsabs.harvard.edu/abs/2011BSRSL..8...,...,4.768240e-11,2.207940e-14,0.696430,0.002302,0.695320,0.001003,0.261124,8.67413e-04,-0.174095,0.000870
104,2198,pi Aqr,336.319250,1.377397,4.794,B1Ve,Em*,"Gaia, XMM",gamma Cas type,https://ui.adsabs.harvard.edu/abs/2002ApJ...57...,...,5.126770e-12,1.462180e-14,0.710172,0.004591,0.475989,0.003146,-0.111483,3.09355e-03,-0.393044,0.003631
105,2213,4U 2238+60,339.837083,61.274111,NaN,Be,HMXB,"Gaia, Chandra",Be/XRB ?,https://ui.adsabs.harvard.edu/abs/2000A%26A......,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,2218,EM* MWC 659,341.939042,57.280772,10.150,B0IIIpe,Em*,"Gaia, Swift",unconfirmed,no mentions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
Be_xray_cat[["RAx","DECx"]].to_csv("BeSS_coords.csv",index=False)

## Old method below

### Initialize ViZier query instance

In [2]:
v = Vizier(columns=['all', '_RAJ2000', '_DEJ2000', "_r"])
v.ROW_LIMIT = -1 # no row limit

### Load the [Be star database](http://basebe.obspm.fr/basebe/)

In [3]:
BeSS = ascii_io.read("./catalogs/BeSS_catalog.csv", format="csv", header_start=0, data_start=1)
BeSS["_RAJ2000"] = [hms_to_dec(hms) for hms in BeSS["RA"]]*u.degree
BeSS["_DEJ2000"] = [dms_to_dec(dms) for dms in BeSS["DEC"]]*u.degree
BeSS.pprint()

 #    Be star    Category ...       _RAJ2000            _DEJ2000     
                          ...         deg                 deg        
---- ---------- --------- ... ------------------- -------------------
   1 BD+62 2346 Classical ... 0.35291666666666666   63.50436944444444
   2  HD 224905 Classical ...  0.4109583333333333   60.44992222222222
   3  HD 225095 Classical ...            0.863125  55.550897222222225
   4      2 Cet Classical ...  0.9349583333333333 -17.335991666666665
   5     10 Cas Classical ...  1.6105833333333333   64.19616944444445
   6 BD+59 2829 Classical ...             1.70125  60.600230555555555
   7    BD+62 1 Classical ...  1.8881250000000003  63.080327777777775
   8   BD+62 11 Classical ...              2.6955   63.17293611111111
   9   V742 Cas Classical ...  2.9047916666666667   58.21182777777778
  10   EM* AS 2 Classical ...            3.246875   66.32206388888889
 ...        ...       ... ...                 ...                 ...
2255   V817 Cas Clas

### Crossmatch with Gaia data

Create `gaia_df`, a `pandas.DataFrame` containing the single closest match (within $1^{\prime\prime}$) in the Gaia catalog for every entry in the BeSS database.

In [4]:
Gaia_BeSS_1as = v.query_region(BeSS, catalog="I/350/gaiaedr3", radius="1s")
Gaia_matched = Gaia_BeSS_1as[0]["_q","_r","EDR3Name","Plx","pmRA","pmDE","epsi","sepsi","Gmag","BPmag","RPmag","RVDR2","GLON","GLAT","Tefftemp"]

gaia_df = Gaia_matched.to_pandas()
for i in BeSS["#"]:
    instances = len(gaia_df[gaia_df["_q"]==i])
    if instances == 0:
        pass
    elif instances >= 2:
        dupeframe = gaia_df[gaia_df["_q"]==i][["_q","_r"]]
        gaia_df.drop(dupeframe[dupeframe["_r"]>min(dupeframe["_r"])].index, inplace=True)

gaia_df.dropna(axis=0, subset=["_r"], inplace=True)
F0 = 2.9979246e-5 * 3631 / (6217.59**2) # Jy -> erg/cm^2/s/Ang
gaia_df["Gflux"] = F0 * 10**(gaia_df["Gmag"]/-2.5)

assert(len(gaia_df) == len(np.unique(gaia_df["_q"]))),"Something went wrong..."

### HMXB crossmatch

- 2CXO J210335.6+454505 with 92%/100% is a HMXB in our TD
- 2CXO J213930.6+565910 with 83%/99% is a HMXB from Simbad, but not in our TD
- 2CXO J112057.1-615500 with 96%/100% is a HMXB from Liu+06 in our TD
- 2CXO J223920.8+611626 with 81%/100% is a HMXB from Liu+06 in our TD
- 2CXO J174445.7-271344 #  not significant with 45%/70% is a HMXB in our TD, but a Be Star in Simbad
- 2CXO J024031.6+611345 #  not significant with 58%/95% is a HMXB from Liu+06
- 2CXO J113106.9-625648 #  not significant with 36%/35% is not in our TD, is a HMXB from Simbad
- 2CXO J130247.6-635008 #  not significant with 53%/56% is not in our TD, is a HMXB from Simbad, 
- 2CXO J134632.5-625523 #  not significant with 44%/47% is not in our TD, is a Be Star from Simbad

In [6]:
HMXBs_BeSS_1as = v.query_region(BeSS, catalog="J/A+A/455/1165/table1", radius="1s")

### XMM, Chandra, and Swift/XRT catalogs

| Band      | Energy bin range |
|-----------|------------------|
| XMM Flux1 | 0.2 - 0.5 keV    |
| XMM Flux2 | 0.5 - 1.0 keV    |
| XMM Flux3 | 1.0 - 2.0 keV    |
| XMM Flux4 | 2.0 - 4.5 keV    |
| XMM Flux5 | 4.5 - 12.0 keV   |
| XMM Flux8 | 0.2 - 12.0 keV   |
| XMM Flux9 | 0.5 - 4.5 keV    |
| CXO Fluxu | 0.2 - 0.5 keV    |
| CXO Fluxs | 0.5 - 1.2 keV    |
| CXO Fluxm | 1.2 - 2.0 keV    |
| CXO Fluxh | 2.0 - 7.0 keV    |
| CXO Fluxb | 0.5 - 7.0 keV    |

Load the full catalogs from locally stored copies. Crossmatch from vizier remotely.

In [51]:
cat_ids = ["IX/59/xmm4dr9s","IX/57/csc2master","IX/58/2sxpscle"]
Xmatch = [None, None, None]
radii = ["2s","1s","5s"]
for i in range(len(cat_ids)):
    Xmatch[i] = v.query_region(BeSS, catalog=cat_ids[i], radius=radii[i])
    print("Successfully matched BeSS with",cat_ids[i])
[XMM_xmatch, CSC_xmatch, XRT_xmatch] = Xmatch

Successfully matched BeSS with IX/59/xmm4dr9s
Successfully matched BeSS with IX/57/csc2master
Successfully matched BeSS with IX/58/2sxpscle


In [10]:
Chandra_sources = CSC_xmatch[0]["_q","_2CXO","Fluxb","Fluxh","Fluxm","Fluxs","Fluxu","HRhm","HRhs","HRms",
                                "VaKSb","VaKPb","VaGLb","Vib"]
chandra_bands = {"b":(0.5,7), "h":(2,7), "m":(1.2,2), "s":(0.5,1.2), "u":(0.2,0.5)}

XMM_sources = XMM_xmatch[0]["_q","_4XMM","Flux1","Flux2","Flux3","Flux4","Flux5","Flux8","Flux9",
                            "HR1","HR2","HR3","HR4","Cst","Fvar","e_Fvar","V"]
xmm_bands = dict(zip(list("1234589"),[(0.2,0.5),(0.5,1.0),(1.0,2.0),(2.0,4.5),(4.5,12.0),(0.2,12.0),(0.5,4.5)]))

XRT_sources = XRT_xmatch[0]["_q","_2SXPS","HR1","HR2","FPO0","FPU0","NH1H","Gamma"]

### Compile a master catalog

In [11]:
objs = pd.merge(gaia_df, HMXBs_BeSS_1as[0].to_pandas(), on="_q", how="outer")
xray = pd.merge(XMM_sources.to_pandas(), Chandra_sources.to_pandas(), on="_q", how="outer")
xray = pd.merge(xray, XRT_sources.to_pandas(), on="_q", how="outer")

master = pd.merge(BeSS.to_pandas(), pd.merge(objs, xray, on="_q", how="outer"), left_on="#", right_on="_q", how="outer")

In [12]:
for i in master.index:
    flags = []
    if pd.notna(master.loc[i,"Be star"]):
        flags.append("BeSS")
    if pd.notna(master.loc[i,"EDR3Name"]):
        flags.append("Gaia")
    if pd.notna(master.loc[i,"Name"]):
        flags.append("HMXB")
    if pd.notna(master.loc[i,"_4XMM"]):
        flags.append("XMM")
    if pd.notna(master.loc[i,"_2CXO"]):
        flags.append("Chandra")
    master.loc[i,"flags"] = ",".join(flags)

In [13]:
master.drop(['RA','DEC','_r_x','_RAJ2000_y','_DEJ2000_y','_r_y','RAJ2000','DEJ2000','GLON_y','GLAT_y',"u_Name2"], axis=1, inplace=True)
master.rename(columns={"V_x":"V","Type_x":"SpecType",'_RAJ2000_x':"_RAJ2000",'_DEJ2000_x':"_DEJ2000","GLON_x":"GLON","GLAT_x":"GLAT","Type_y":"Type","V_y":"Vflag"}, inplace=True)

In [15]:
master.columns

Index(['#', 'Be star', 'Category', 'V', 'SpecType', 'vsini',
       'Nb of spectra in BeSS', '_RAJ2000', '_DEJ2000', '_q', 'EDR3Name',
       'Plx', 'pmRA', 'pmDE', 'epsi', 'sepsi', 'Gmag', 'BPmag', 'RPmag',
       'RVDR2', 'GLON', 'GLAT', 'Tefftemp', 'Gflux', 'Name', 'Type', 'Pos',
       'Opt', 'r_Opt', 'l_Vmag', 'Vmag', 'Vmagl', 'u_Vmag', 'B-V', 'u_B-V',
       'B-Vl', 'U-B', 'E_B-V_', 'E_B-V_2', 'u_E_B-V_', 'r_Vmag', 'Jmag',
       'l_Hmag', 'Hmag', 'l_Kmag', 'Kmag', 'l_FX', 'FX', 'FXu', 'Range',
       'r_FX', 'Porb', 'u_Porb', 'Ppulse', 'u_Ppulse', 'r_Porb', 'SpType',
       'Name2', 'Name3', 'Name4', 'Simbad', '_4XMM', 'Flux1', 'Flux2', 'Flux3',
       'Flux4', 'Flux5', 'Flux8', 'Flux9', 'HR1_x', 'HR2_x', 'HR3', 'HR4',
       'Cst', 'Fvar', 'e_Fvar', 'Vflag', '_2CXO', 'Fluxb', 'Fluxh', 'Fluxm',
       'Fluxs', 'Fluxu', 'HRhm', 'HRhs', 'HRms', 'VaKSb', 'VaKPb', 'VaGLb',
       'Vib', '_2SXPS', 'HR1_y', 'HR2_y', 'FPO0', 'FPU0', 'NH1H', 'Gamma',
       'flags'],
      dtype='object

In [41]:
#master.to_csv("./catalogs/BeSS+Gaia+HMXBs+CSC+XMM+XRT.csv", index=False)